In [ ]:
from databricks.sdk import WorkspaceClient
import pandas as pd

In [ ]:
def query_genie(genie_space_id:str, user_question:str):
  w = WorkspaceClient()
  # initialize output
  output = {
        'user_question': user_question,
        'genie_status': None,
        'generated_sql': None,
        'query_description': None,
        'sql_status': None,
        'sql_result': None,
        'error': None
    }
  
  # Query Genie space
  conversation = w.genie.start_conversation(space_id = genie_space_id , content = user_question)
  message_waiter = w.genie.create_message(space_id = genie_space_id, conversation_id = conversation.conversation_id, content = user_question)
  genie_result = message_waiter.result()
  output['genie_status'] = genie_result.status.value # update output (genie status)

  # Check if Genie completed
  if genie_result.status.value != 'COMPLETED':
      output['error'] = f"Genie workflow did not complete. Status: {genie_result.status.value}" # update output (error)
      return output

  # Check for attachments
  if not genie_result.attachments:
      output['error'] = 'No atatchments found in Genie response' # update output (error)
      return output

  # Extract query and results
  query_found = False
  for attachment in genie_result.attachments:
      if attachment.query:
          query_found = True
          output['generated_sql'] = attachment.query.query # update output (generated sql)
          output['query_description'] = attachment.query.description if hasattr(attachment.query,'description') else None # the request being sent to Genie, but not the action plan

          statement_id = attachment.query.statement_id

          # fetch query results
          statement_result = w.statement_execution.get_statement(statement_id)

          # Get SQL execution status
          output['sql_status'] = statement_result.status.state.value # update output (sql status)

          # Check if SQL execution succeded
          if output['sql_status'] != 'SUCCEDDED':
              output['error'] = f"SQL execution failed with status:{output['sql_status']}" # update output (error)
        
          # extract results
          if statement_result.result and statement_result.result.data_array:
              result_data = statement_result.result.data_array

              # Get column names if available
              column_names = None
              if statement_result.manifest and statement_result.manifest.schema:
                  column_names = [col.name for col in statement_result.manifest.schema.columns]
              
              # Format as data frame
              if column_names:
                  output['sql_result'] = pd.DataFrame(result_data, columns = column_names)
              else:
                  output['sql_result'] = pd.DataFrame(result_data)
          
          else:
              output['error'] = 'Query succeeded but returned no data'
          
          break
  
  if not query_found:
      output['error'] = 'No SQL query found in attachments'

  return output     



In [ ]:
genie_space_id = 'abc'
user_question = 'abc'
query_genie(genie_space_id = genie_space_id, user_question = user_question)